In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from datetime import datetime

In [2]:
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')

In [3]:
spark = SparkSession.builder\
    .appName('autism-characteristics')\
    .config(conf=conf)\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4236ad17-85de-4b7a-a26c-83a318c277db;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 316ms :: artifacts dl 8ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------------------

22/10/13 23:48:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.parquet("s3a://projeto-pi-datalake-trusted/autism-region")

22/10/13 23:49:06 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
df.printSchema()

root
 |-- id_paciente: string (nullable = true)
 |-- idade_atual: string (nullable = true)
 |-- idade_descoberta: string (nullable = true)
 |-- genero: string (nullable = true)
 |-- grau: string (nullable = true)
 |-- sensibilidade_sentidos: string (nullable = true)
 |-- agressivo: string (nullable = true)
 |-- hiperativo: string (nullable = true)
 |-- movimentos_repetitivos: string (nullable = true)
 |-- baixa_concentracao: string (nullable = true)
 |-- hiperfoco: string (nullable = true)
 |-- necessidade_rotina: string (nullable = true)
 |-- dificuldade_imaginacao: string (nullable = true)
 |-- introvertido: string (nullable = true)
 |-- tipo_autismo: string (nullable = true)
 |-- tipo_autismo_descricao: string (nullable = true)
 |-- classe: string (nullable = true)
 |-- classe-descricao: string (nullable = true)
 |-- empregado: boolean (nullable = true)
 |-- per_capita: double (nullable = true)
 |-- quantidade_familia: long (nullable = true)
 |-- total_salario: long (nullable = true

In [6]:
df.head()

22/10/12 00:03:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(id_paciente='8647DC55C75FE6B49B05AB4F6E2C4EF604EFE2672FC477BB76C0114F8F', idade_atual='44', idade_descoberta='29', genero='B850D4F5937B17CE7D2632EE42F6F88AFE6BCAE572821F2612C20B0CD3', grau='severo', sensibilidade_sentidos='True', agressivo='True', hiperativo='False', movimentos_repetitivos='False', baixa_concentracao='True', hiperfoco='True', necessidade_rotina='True', dificuldade_imaginacao='True', introvertido='False', tipo_autismo='Transtorno do espectro Autista (TEA)', tipo_autismo_descricao='Está extremidade mais branda do espectro autista, pois a inteligência pode ser alta e a capacidade de realizar as atividades diárias é preservada. No entanto, a dificuldade na interação social é muito comum.', classe='B', classe-descricao='Baixa classe alta', empregado=True, per_capita=9977.0, quantidade_familia=1, total_salario=9977, PIB_nacional=8700000000000, PIB_percent_BR=5.3, PIB_regional=1195000000000, area_regiao=3853676948, densidade_demografica_regiao=4.12, gentilico_regiao='nort

In [7]:
execution_date = datetime(2022, 6, 1)

In [8]:
df_filtred = (
    df.select(
        "tipo_autismo", "classe", "classe-descricao", "tipo_autismo_descricao", "grau", "genero", "empregado",
        "dificuldade_imaginacao", "necessidade_rotina", "hiperfoco", "baixa_concentracao", "movimentos_repetitivos",
        "hiperativo", "agressivo", "sensibilidade_sentidos", "introvertido"
    )
    .withColumn('year', f.lit(execution_date.year))
    .withColumn('month', f.lit(execution_date.month))
    .withColumn('day', f.lit(execution_date.day))
)

In [9]:
df_filtred.printSchema()

root
 |-- tipo_autismo: string (nullable = true)
 |-- classe: string (nullable = true)
 |-- classe-descricao: string (nullable = true)
 |-- tipo_autismo_descricao: string (nullable = true)
 |-- grau: string (nullable = true)
 |-- genero: string (nullable = true)
 |-- empregado: boolean (nullable = true)
 |-- dificuldade_imaginacao: string (nullable = true)
 |-- necessidade_rotina: string (nullable = true)
 |-- hiperfoco: string (nullable = true)
 |-- baixa_concentracao: string (nullable = true)
 |-- movimentos_repetitivos: string (nullable = true)
 |-- hiperativo: string (nullable = true)
 |-- agressivo: string (nullable = true)
 |-- sensibilidade_sentidos: string (nullable = true)
 |-- introvertido: string (nullable = true)
 |-- year: integer (nullable = false)
 |-- month: integer (nullable = false)
 |-- day: integer (nullable = false)



In [10]:
df_filtred.write.mode('overwrite').partitionBy('year', 'month', 'day')\
    .csv('s3a://projeto-pi-datalake-analytics/autism-characteristics', header=True)

22/10/12 00:03:49 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


22/10/12 00:04:32 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/10/12 00:04:32 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [5]:
# secret = 'aYszPMHCQhrbqyv3BaqtW7yufaf7I0Up'

In [6]:
# df.select(f.expr(f"decode(aes_decrypt(unhex(id_paciente), '{secret}'), 'UTF-8')"), "id_paciente").head()

Row(decode(aes_decrypt(unhex(id_paciente), aYszPMHCQhrbqyv3BaqtW7yufaf7I0Up, GCM, DEFAULT), UTF-8)='2', id_paciente='8647DC55C75FE6B49B05AB4F6E2C4EF604EFE2672FC477BB76C0114F8F')

In [13]:
spark.sparkContext.stop()
spark.stop()